In [1]:
import torch
import torchvision 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [17]:
import PIL
from PIL import Image

In [2]:
import os 
import cv2
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
import glob

In [3]:
from torchvision import datasets, transforms
from torch.autograd import Variable

In [91]:
class CustomDataset(Dataset):
    def __init__(self):
        self.imgs_path = 'gan_data/'
        file_list = glob.glob(self.imgs_path + "*")
        self.data=[]
        for class_path in file_list:
            for img_path,img_H,img_pose in zip(glob.glob(class_path + "/*.jpg"),glob.glob(class_path + "/*).npy"),glob.glob(class_path + "/*pose.npy")):
                for img1_path,img1_H,img1_pose in zip(glob.glob(class_path + "/*.jpg"),glob.glob(class_path + "/*).npy"),glob.glob(class_path + "/*pose.npy")):
                    if img_path != img1_path :
                        self.data.append([img_path,img_H,img_pose,img1_path,img1_H,img1_pose])
        self.img_dim = (72,128)
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        img_path,z_path,img_pose_path,img1_path,z1_path,img1_pose_path = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img,self.img_dim)
        img_tensor = torch.from_numpy(img)
        z_ = np.load(z_path)
        z_= z_.astype(np.float32)
        
        z_tensor = torch.from_numpy(z_)
        img_pose = np.load(img_pose_path)
        img_pose_tensor = torch.from_numpy(img_pose)
        
        img1 = cv2.imread(img1_path)
        img1 = cv2.resize(img1,self.img_dim)
        img1_tensor = torch.from_numpy(img1)
        z1_ = np.load(z1_path)
        z1_= z1_.astype(np.float32)
        z1_tensor = torch.from_numpy(z1_)
        img1_pose = np.load(img1_pose_path)
        img1_pose_tensor = torch.from_numpy(img1_pose)
        return img_tensor,z_tensor,img_pose_tensor,img1_tensor,z1_tensor,img1_pose_tensor

In [78]:
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [92]:
dataset = CustomDataset()
data_loader = DataLoader(dataset,batch_size = 16)

In [82]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.conv1  = nn.Conv2d(4,16,3)
        self.conv2  = nn.Conv2d(16,32,3)
        self.conv3  = nn.Conv2d(32,48,3)
        self.conv4  = nn.Conv2d(48,64,3)
        
        self.convT1 = nn.ConvTranspose2d(64,48,3)
        self.convT2 = nn.ConvTranspose2d(48,32,3)
        self.convT3 = nn.ConvTranspose2d(32,16,3)
        self.convT4 = nn.ConvTranspose2d(16,4,2)
        self.convT5 = nn.ConvTranspose2d(4,3,2)
        
        self.conv1_1  = nn.Conv2d(1,3,2)
        self.conv2_1 = nn.Conv2d(3,6,2)
        self.conv3_1 = nn.ConvTranspose2d(6,6,2)
        self.conv4_1 = nn.ConvTranspose2d(6,9,2)
        
        
        self.conv1_2 = nn.Conv2d(12,10,3)
        self.conv2_2 = nn.Conv2d(10,8,3)
        self.conv3_2 = nn.Conv2d(8,6,3)
        self.conv1T_1  = nn.ConvTranspose2d(6,4,3)
        self.conv2T_1 = nn.ConvTranspose2d(4,3,3)
        self.conv3T_1 = nn.ConvTranspose2d(3,3,3)
        
        
    def forward(self,x) :
        
        x1,x2 = x
        x1 =x1.to(device)
        x2 = x2.to(device)
        x1 = x1.permute(0,3,1, 2)
        
        x1 = F.relu(self.conv1(x1))
        x1_1 = F.relu(self.conv2(x1))
        x1_2 = F.relu(self.conv3(x1_1))
        x1 = F.relu(self.conv4(x1_2))
        
        x1 = F.relu(self.convT1(x1))
        x1 = F.relu(self.convT2(x1))
        x1 = F.relu(self.convT3(x1))
        x1 = F.relu(self.convT4(x1))
        x1 = F.relu(self.convT5(x1))
        
        
        x2 = x2.permute(0,3,1,2)
        x2 = F.relu(self.conv1_1(x2))
        x2 = F.relu(self.conv2_1(x2))
        x2 = F.relu(self.conv3_1(x2))
        x2 = F.relu(self.conv4_1(x2))
        
        
        x1 = x1.permute(0,2,3,1)
        x2 =x2.permute(0,2,3,1)
        x3 = torch.concat((x1,x2),axis=-1)
        x3 = x3.permute(0,3,1, 2)
        
        x3 = F.relu(self.conv1_2(x3))
        x3 = F.relu(self.conv2_2(x3))
        x3 = F.relu(self.conv3_2(x3))
        x3 = F.relu(self.conv1T_1(x3))
        x3 = F.relu(self.conv2T_1(x3))
        x3 = (self.conv3T_1(x3))
        x3 = x3.permute(0,2,3,1)
        

        
        return x3

In [83]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.conv1  = nn.Conv2d(3,64,3, bias=False)
        self.conv2  = nn.Conv2d(64,64*2,5, bias=False)
        self.conv3  = nn.Conv2d(64*2,1,10, bias=False)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(30,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x) :
        x = x.permute(0,3,1,2)
        x = F.relu(self.pool((self.conv1(x))))
        x = F.relu(self.pool((self.conv2(x))))
        x = F.relu(self.pool((self.conv3(x))))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.flatten(x,1)
        
        
        return x

In [84]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [85]:
adv_loss = nn.BCEWithLogitsLoss()

In [86]:
def nn_loss(predicted, ground_truth, nh=3, nw=3):
    v_pad = nh // 2
    h_pad = nw // 2
    val_pad = nn.ConstantPad2d((v_pad, v_pad, h_pad, h_pad), -10000)(ground_truth)

    reference_tensors = []
    for i_begin in range(0, nh):
        
        i_end = i_begin - nh + 1
        i_end = None if i_end == 0 else i_end
        for j_begin in range(0, nw):
            j_end = j_begin - nw + 1
            j_end = None if j_end == 0 else j_end
            sub_tensor = val_pad[:, :, i_begin:i_end, j_begin:j_end]
            reference_tensors.append(sub_tensor.unsqueeze(-1))
    reference = torch.cat(reference_tensors, dim=-1)
    ground_truth = ground_truth.unsqueeze(dim=-1)

    predicted = predicted.unsqueeze(-1)
    abs = torch.abs(reference - predicted)
    norms = torch.sum(abs, dim=1)
    loss,_ = torch.min(norms, dim=-1)
    loss = torch.mean(loss)

    return loss

In [87]:
def gen_loss(fake_img,input_):
    return adv_loss(fake_img,input_)
def disc_loss(fake_img,target_img):
    return adv_loss(fake_img,target_img+0.0001)

In [88]:
learning_rate = 0.0001
G_optimizer = optim.Adam(generator.parameters(), lr = learning_rate)
D_optimizer = optim.Adam(discriminator.parameters(), lr = learning_rate)


In [96]:
num_epochs =100
gen_losss=[]
disc_losss=[]

for epochs in range(1,num_epochs+1):
    for index, (real_img,real_img_noise,real_img_pose,target_img,target_img_noise,target_img_pose) in enumerate(data_loader):
        D_optimizer.zero_grad()
        
        # Discriminator training ----
        real_img = real_img.float()
        real_img = real_img.to(device)
        
        target_img = target_img.float()
        target_img = target_img.to(device)
        target_img_noise = target_img_noise.to(device)
        
        real_img_noise = real_img_noise.float().to(device)
        real_target = Variable(torch.ones(real_img.size(0)).to(device))
        real_target = real_target.unsqueeze(1)

        fake_target = Variable(torch.zeros(real_img.size(0)).to(device))
        fake_target = fake_target.unsqueeze(1)
        
        output = discriminator(real_img)
        D_real_loss = disc_loss(output,real_target)
        D_real_loss.backward()
        
        
        real_noise_vector = torch.concat((real_img.reshape(real_img.shape[0],128,72,3),real_img_noise.reshape(real_img.shape[0],128,72,1)),axis=-1).to(device)
        target_noise_vector = target_img_noise.reshape(real_img.shape[0],128,72,1)
        target_noise_vector = target_noise_vector.to(device)
        
        generated_img = generator([real_noise_vector,target_noise_vector])
        output = discriminator(generated_img.detach())
        D_fake_loss = disc_loss(output,fake_target)
        D_fake_loss.backward()
        D_optimizer.step()
        
        if index%100 == 0:
            print(f'disc_loss={D_fake_loss.item() + D_real_loss.item()}')
        disc_losss.append(D_fake_loss.item() + D_real_loss.item())
        # Generator training ----
        G_optimizer.zero_grad()
        gen_output = discriminator(generated_img)
        G_loss = gen_loss(gen_output,real_target)*10 + nn_loss(generated_img,target_img)
        
        if index%100 == 0:
            print(f'gen_loss={G_loss.item()}')
        gen_losss.append(G_loss.item())
        G_loss.backward()
        G_optimizer.step()
        
        if index%1000 == 0:
            for i in range(real_img.shape[0]):
                x=generated_img[i].detach().to('cpu').reshape(128,72,3)
                x=x.numpy()
                im =Image.fromarray(x.astype(np.uint8))
                im.save(f'gen_{i}.jpeg')
                x_=target_img[i].detach().to('cpu').reshape(128,72,3)
                x_=x_.numpy()
                im =Image.fromarray(x_.astype(np.uint8))
                im.save(f'target_{i}.jpeg')
            
    print(f'{epochs} done')

disc_loss=1.3862944841384888
gen_loss=4429.37890625
disc_loss=1.3862944841384888
gen_loss=2802.86181640625
disc_loss=1.3862944841384888
gen_loss=2829.01416015625
disc_loss=1.3862944841384888
gen_loss=2526.551513671875
disc_loss=1.3862944841384888
gen_loss=3030.526611328125
disc_loss=1.3862944841384888
gen_loss=2441.906005859375
disc_loss=1.3862944841384888
gen_loss=2066.56591796875
disc_loss=1.3862944841384888
gen_loss=2257.173095703125
disc_loss=1.3862944841384888
gen_loss=2981.0068359375
disc_loss=1.3862944841384888
gen_loss=2969.208740234375
disc_loss=1.3862944841384888
gen_loss=2582.1220703125
disc_loss=1.3862944841384888
gen_loss=2684.03515625
disc_loss=1.3862944841384888
gen_loss=4260.8623046875
disc_loss=1.3862944841384888
gen_loss=4001.2177734375
disc_loss=1.3862944841384888
gen_loss=4075.360107421875
disc_loss=1.3862944841384888
gen_loss=3482.011962890625
disc_loss=1.3862944841384888
gen_loss=4093.48681640625
disc_loss=1.3862944841384888
gen_loss=4317.00244140625
disc_loss=1.3

disc_loss=1.3862944841384888
gen_loss=3133.770263671875
disc_loss=1.3862944841384888
gen_loss=3088.282470703125
disc_loss=1.3862944841384888
gen_loss=2998.172607421875
disc_loss=1.3862944841384888
gen_loss=2797.470947265625
disc_loss=1.3862944841384888
gen_loss=2631.1337890625
disc_loss=1.3862944841384888
gen_loss=3358.000732421875
disc_loss=1.3862944841384888
gen_loss=3746.666748046875
disc_loss=1.3862944841384888
gen_loss=3645.777099609375
disc_loss=1.3862944841384888
gen_loss=3338.301513671875
disc_loss=1.3862944841384888
gen_loss=3760.599609375
disc_loss=1.3862944841384888
gen_loss=4827.81591796875
disc_loss=1.3862944841384888
gen_loss=3119.73388671875
disc_loss=1.3862944841384888
gen_loss=3247.697998046875
disc_loss=1.3862944841384888
gen_loss=2703.844970703125
disc_loss=1.3862944841384888
gen_loss=2957.7060546875
disc_loss=1.3862944841384888
gen_loss=2824.035888671875
disc_loss=1.3862944841384888
gen_loss=2733.005859375
disc_loss=1.3862944841384888
gen_loss=2620.947509765625
disc

disc_loss=1.3862944841384888
gen_loss=3146.027099609375
disc_loss=1.3862944841384888
gen_loss=2575.02099609375
disc_loss=1.3862944841384888
gen_loss=2249.943603515625
disc_loss=1.3862944841384888
gen_loss=3450.57470703125
disc_loss=1.3862944841384888
gen_loss=1964.6302490234375
disc_loss=1.3862944841384888
gen_loss=3437.148193359375
disc_loss=1.3862944841384888
gen_loss=3291.604248046875
disc_loss=1.3862944841384888
gen_loss=2461.946044921875
disc_loss=1.3862944841384888
gen_loss=2446.657958984375
disc_loss=1.3862944841384888
gen_loss=3543.099365234375
disc_loss=1.3862944841384888
gen_loss=2790.135986328125
disc_loss=1.3862944841384888
gen_loss=3317.078857421875
disc_loss=1.3862944841384888
gen_loss=2560.051025390625
disc_loss=1.3862944841384888
gen_loss=2456.556396484375
disc_loss=1.3862944841384888
gen_loss=3272.052490234375
disc_loss=1.3862944841384888
gen_loss=2525.427734375
disc_loss=1.3862944841384888
gen_loss=2495.2734375
disc_loss=1.3862944841384888
gen_loss=2884.618408203125
d

disc_loss=1.3862944841384888
gen_loss=2397.712158203125
disc_loss=1.3862944841384888
gen_loss=2477.85546875
disc_loss=1.3862944841384888
gen_loss=2798.00244140625
disc_loss=1.3862944841384888
gen_loss=2173.369384765625
disc_loss=1.3862944841384888
gen_loss=2426.4765625
disc_loss=1.3862944841384888
gen_loss=2548.573486328125
disc_loss=1.3862944841384888
gen_loss=2431.593017578125
disc_loss=1.3862944841384888
gen_loss=2198.91015625
disc_loss=1.3862944841384888
gen_loss=2353.853759765625
disc_loss=1.3862944841384888
gen_loss=2383.2783203125
disc_loss=1.3862944841384888
gen_loss=2357.01318359375
disc_loss=1.3862944841384888
gen_loss=2419.1748046875
disc_loss=1.3862944841384888
gen_loss=2027.186279296875
disc_loss=1.3862944841384888
gen_loss=1836.8018798828125
disc_loss=1.3862944841384888
gen_loss=1884.137451171875
disc_loss=1.3862944841384888
gen_loss=1717.1053466796875
disc_loss=1.3862944841384888
gen_loss=1687.062255859375
disc_loss=1.3862944841384888
gen_loss=4554.2666015625
disc_loss=1

disc_loss=1.3862944841384888
gen_loss=1532.25732421875
disc_loss=1.3862944841384888
gen_loss=2033.7642822265625
disc_loss=1.3862944841384888
gen_loss=1835.9925537109375
disc_loss=1.3862944841384888
gen_loss=2263.30712890625
disc_loss=1.3862944841384888
gen_loss=1797.321533203125
disc_loss=1.3862944841384888
gen_loss=1597.3521728515625
disc_loss=1.3862944841384888
gen_loss=2001.9775390625
disc_loss=1.3862944841384888
gen_loss=1460.7662353515625
disc_loss=1.3862944841384888
gen_loss=2101.1005859375
disc_loss=1.3862944841384888
gen_loss=1767.496826171875
disc_loss=1.3862944841384888
gen_loss=2123.029296875
disc_loss=1.3862944841384888
gen_loss=2100.29736328125
disc_loss=1.3862944841384888
gen_loss=1558.799560546875
disc_loss=1.3862944841384888
gen_loss=1941.1302490234375
disc_loss=1.3862944841384888
gen_loss=1705.0989990234375
disc_loss=1.3862944841384888
gen_loss=2123.8779296875
disc_loss=1.3862944841384888
gen_loss=1930.560791015625
disc_loss=1.3862944841384888
gen_loss=2149.46069335937

...

disc_loss=1.3862944841384888
gen_loss=2255.593994140625
disc_loss=1.3862944841384888
gen_loss=2375.982666015625
disc_loss=1.3862944841384888
gen_loss=2319.4189453125
disc_loss=1.3862944841384888
gen_loss=1994.67724609375
disc_loss=1.3862944841384888
gen_loss=2462.14599609375
disc_loss=1.3862944841384888
gen_loss=2001.0335693359375
disc_loss=1.3862944841384888
gen_loss=2262.9658203125
disc_loss=1.3862944841384888
gen_loss=2099.5576171875
disc_loss=1.3862944841384888
gen_loss=2223.06884765625
disc_loss=1.3862944841384888
gen_loss=2108.89306640625
disc_loss=1.3862944841384888
gen_loss=2275.317138671875
disc_loss=1.3862944841384888
gen_loss=2683.16845703125
disc_loss=1.3862944841384888
gen_loss=2012.830810546875
disc_loss=1.3862944841384888
gen_loss=1945.6439208984375
disc_loss=1.3862944841384888
gen_loss=2138.762451171875
disc_loss=1.3862944841384888
gen_loss=1712.648193359375
disc_loss=1.3862944841384888
gen_loss=2163.630615234375
disc_loss=1.3862944841384888
gen_loss=2154.3212890625
dis

disc_loss=1.3862944841384888
gen_loss=2099.059326171875
disc_loss=1.3862944841384888
gen_loss=1967.9736328125
disc_loss=1.3862944841384888
gen_loss=1853.742431640625
disc_loss=1.3862944841384888
gen_loss=1792.2557373046875
disc_loss=1.3862944841384888
gen_loss=1772.2427978515625
disc_loss=1.3862944841384888
gen_loss=1853.0316162109375
disc_loss=1.3862944841384888
gen_loss=3403.8984375
disc_loss=1.3862944841384888
gen_loss=3175.9365234375
disc_loss=1.3862944841384888
gen_loss=2187.826904296875
disc_loss=1.3862944841384888
gen_loss=2172.958740234375
disc_loss=1.3862944841384888
gen_loss=1967.9765625
disc_loss=1.3862944841384888
gen_loss=2301.63134765625
disc_loss=1.3862944841384888
gen_loss=2017.80419921875
disc_loss=1.3862944841384888
gen_loss=1875.6195068359375
disc_loss=1.3862944841384888
gen_loss=1875.9678955078125
disc_loss=1.3862944841384888
gen_loss=1752.869140625
disc_loss=1.3862944841384888
gen_loss=2693.87060546875
disc_loss=1.3862944841384888
gen_loss=2139.67724609375
disc_los

disc_loss=1.3862944841384888
gen_loss=3286.955810546875
7 done
disc_loss=1.3862944841384888
gen_loss=3830.222412109375
disc_loss=1.3862944841384888
gen_loss=2721.080322265625
disc_loss=1.3862944841384888
gen_loss=2688.5029296875
disc_loss=1.3862944841384888
gen_loss=2379.95947265625
disc_loss=1.3862944841384888
gen_loss=2938.74462890625
disc_loss=1.3862944841384888
gen_loss=2314.39306640625
disc_loss=1.3862944841384888
gen_loss=2119.509033203125
disc_loss=1.3862944841384888
gen_loss=2072.845458984375
disc_loss=1.3862944841384888
gen_loss=2865.54638671875
disc_loss=1.3862944841384888
gen_loss=3009.412109375
disc_loss=1.3862944841384888
gen_loss=2474.664306640625
disc_loss=1.3862944841384888
gen_loss=2578.214111328125
disc_loss=1.3862944841384888
gen_loss=3663.28076171875
disc_loss=1.3862944841384888
gen_loss=3609.757568359375
disc_loss=1.3862944841384888
gen_loss=3942.216552734375
disc_loss=1.3862944841384888
gen_loss=3225.145263671875
disc_loss=1.3862944841384888
gen_loss=3547.85424804

disc_loss=1.3862944841384888
gen_loss=2600.494873046875
disc_loss=1.3862944841384888
gen_loss=2470.797119140625
disc_loss=1.3862944841384888
gen_loss=2638.4140625
disc_loss=1.3862944841384888
gen_loss=2547.58251953125
disc_loss=1.3862944841384888
gen_loss=2341.808837890625
disc_loss=1.3862944841384888
gen_loss=2199.83349609375
disc_loss=1.3862944841384888
gen_loss=2865.492431640625
disc_loss=1.3862944841384888
gen_loss=2539.0205078125
disc_loss=1.3862944841384888
gen_loss=2364.097900390625
disc_loss=1.3862944841384888
gen_loss=2510.4111328125
disc_loss=1.3862944841384888
gen_loss=3426.41162109375
disc_loss=1.3862944841384888
gen_loss=4415.13525390625
disc_loss=1.3862944841384888
gen_loss=2477.814697265625
disc_loss=1.3862944841384888
gen_loss=2774.214111328125
disc_loss=1.3862944841384888
gen_loss=2271.382568359375
disc_loss=1.3862944841384888
gen_loss=2477.66259765625
disc_loss=1.3862944841384888
gen_loss=2229.3505859375
disc_loss=1.3862944841384888
gen_loss=2205.607421875
disc_loss=1

KeyboardInterrupt: 